In [1]:
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 3.0 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=b18c6ffab9dad2f95496798560a060a5ed0958b415d0830d38bfd0dbe9ef8287
  Stored in directory: /root/.cache/pip/wheels/8b/67/f4/2caaae2146198dcb824f31a303833b07b14a5ec863fb3acd7b
Successfully built autograd-gamma


In [2]:
import numpy as np  # Importing NumPy for numerical operations
import pandas as pd  # Importing Pandas for data manipulation
import matplotlib.pyplot as plt  # Importing Matplotlib for plotting
from lifelines import KaplanMeierFitter  # Importing Kaplan-Meier estimator from lifelines
from lifelines.statistics import logrank_test  # Importing log-rank test for survival comparison

In [3]:
from google.colab import files
uploaded= files.upload()

Saving GBM_tumor_sizes_data.csv to GBM_tumor_sizes_data.csv


In [4]:
pd_data= pd.read_csv('/content/GBM_tumor_sizes_data.csv')
pd_data.head()

,age_months,drug_candidate,tumor_volume_mm2
0,0~6,placebo+TMZ,1611.760684
1,0~6,placebo+TMZ,1468.890532
2,0~6,placebo+TMZ,1645.729921
3,0~6,placebo+TMZ,1842.681718
4,0~6,placebo+TMZ,1447.315491


In [6]:
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [8]:
# Regression view of ANOVA: applying OLS to solve for coefficients ( equivalents of means in our case)
### Fit the regression model
model= ols('tumor_volume_mm2~ C(age_months)+ C(drug_candidate)+ C(age_months):C(drug_candidate)',data=pd_data).fit()
# perform ANOVA
anova_table= anova_lm(model,typ=2)
anova_table

,sum_sq,df,F,PR(>F)
C(age_months),5.366618e+06,2.0,276.759092,1.022155e-61
C(drug_candidate),1.694063e+07,3.0,582.424396,1.442183e-106
C(age_months):C(drug_candidate),4.440508e+06,6.0,76.333054,8.723920e-52
Residual,2.210567e+06,228.0,NaN,NaN


In [12]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# tukey_drug = pairwise_tukeyhsd(endog=pd_data['tumor_volume_mm2'],
#                                groups=pd_data['drug_candidate'],
#                                alpha=0.05)
# print(tukey_drug)

pd_data['group_combo']= pd_data['age_months'].astype(str)+ '_'+ pd_data['drug_candidate'].astype(str)
tukey_result= pairwise_tukeyhsd(endog=pd_data['tumor_volume_mm2'],
                                              groups= pd_data['group_combo'],
                                alpha=0.05)
print(tukey_result)

               Multiple Comparison of Means - Tukey HSD, FWER=0.05                
      group1            group2       meandiff  p-adj    lower      upper    reject
----------------------------------------------------------------------------------
      0~6_Mi1_TMZ       0~6_Mi2_TMZ    696.936    0.0   594.1073   799.7647   True
      0~6_Mi1_TMZ       0~6_Mi3_TMZ   695.1702    0.0   592.3415   797.9989   True
      0~6_Mi1_TMZ   0~6_placebo+TMZ  1260.3956    0.0  1157.5669  1363.2243   True
      0~6_Mi1_TMZ     18~24_Mi1_TMZ   140.1834 0.0006    37.3547    243.012   True
      0~6_Mi1_TMZ     18~24_Mi2_TMZ   268.7759    0.0   165.9472   371.6046   True
      0~6_Mi1_TMZ     18~24_Mi3_TMZ    251.414    0.0   148.5854   354.2427   True
      0~6_Mi1_TMZ 18~24_placebo+TMZ   534.7997    0.0    431.971   637.6284   True
      0~6_Mi1_TMZ      9~15_Mi1_TMZ   201.5909    0.0    98.7622   304.4195   True
      0~6_Mi1_TMZ      9~15_Mi2_TMZ   393.2785    0.0   290.4498   496.1071   True
    

In [13]:
means = pd_data.groupby('group_combo')['tumor_volume_mm2'].mean().sort_values()
print(means)


group_combo
0~6_Mi1_TMZ           201.062240
18~24_Mi1_TMZ         341.245601
9~15_Mi1_TMZ          402.653095
18~24_Mi3_TMZ         452.476278
18~24_Mi2_TMZ         469.838157
9~15_Mi2_TMZ          594.340697
9~15_Mi3_TMZ          603.790906
18~24_placebo+TMZ     735.861963
0~6_Mi3_TMZ           896.232413
0~6_Mi2_TMZ           897.998222
9~15_placebo+TMZ      996.419203
0~6_placebo+TMZ      1461.457824
Name: tumor_volume_mm2, dtype: float64
